In [1]:
%matplotlib inline

# Calculate electrochemical equilbria

In [12]:
from scipy import optimize

import fipy as fp

In [8]:
Vs = fp.Variable("0.018 l/mol")
R = fp.Variable("1 Nav * kB")
T = fp.Variable("298 K")
Faraday = fp.Variable("1 Nav * e")

simple mesh that only has a cell for electrode and a cell for the electrolyte

In [7]:
mesh = fp.Grid1D(nx=2)

In [30]:
def solid(X, interstitials, substitutionals, solvent):
    """solve for electron and cation concentration, given impurity anion and solvent
    """
    components = interstitials + substitutionals + [solvent]

    subs = 1. - X[0] - X[1]
    for Xj in components:
        subs -= Xj[0].value
        
    charge = 0. + X[0] * interstitials[0].z + X[1] * substitutionals[0].z
    for Xj in components:
        charge += Xj.z * Xj[0].value

    return [subs, charge]

In [71]:
def liquid(X, interstitials, substitutionals, solvent):
    """solve for solvent and anion concentration, given cation and impurity electron
    """
    components = interstitials + substitutionals + [solvent]

    subs = 1. - X[0] - X[1]
    for Xj in components:
        subs -= Xj[1].value
        
    charge = 0. + X[0] * solvent.z + X[1] * substitutionals[1].z
    for Xj in components:
        charge += Xj.z * Xj[1].value

    return [subs, charge]

## divalent cation and salt

### $X_\mathrm{impurity} = 10^{-6}$

In [9]:
impurity = 1e-6

interstitials = [
    fp.CellVariable(mesh=mesh, name="$e^-$", value=[0., impurity])
]

substitutionals = [
    fp.CellVariable(mesh=mesh, name="$M^+2$", value=[0., (Vs * "1 mol/l").inBaseUnits()]),
    fp.CellVariable(mesh=mesh, name="$A^-2$", value=[impurity, 0])
]

N = fp.CellVariable(mesh=mesh, name="$N$", value=[impurity, 0.])

interstitials[0].z   = -1
substitutionals[0].z = +2
substitutionals[1].z = -2
N.z                  = 0

components = interstitials + substitutionals + [N]

In [84]:
interstitials[0][0] = 0.
substitutionals[0][0] = 0.
X0 = [1., .5]
X0 = optimize.fsolve(solid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

interstitials[0][0] = X0[0]
substitutionals[0][0] = X0[1]


N[1] = 0.
substitutionals[1][1] = 0.
X0 = [1., 1.]
X0 = optimize.fsolve(liquid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

N[1] = X0[0]
substitutionals[1][1] = X0[1]

for Xj in components:
    Xj.muSL = fp.numerix.log(Xj[1] / Xj[0])
    print Xj.muSL, Xj.name, Xj

-13.4100424499 $e^-$ [  6.66664667e-01   1.00000000e-06]
-2.91877123242 $M^+$ [ 0.33333333  0.018     ]
9.79809925871 $A^-$ [  1.00000000e-06   1.79995000e-02]
13.7788460549 $N$ [  1.00000000e-06   9.63999500e-01]


## $X_\mathrm{impurity} = 10^{-9}$

In [85]:
impurity = 1e-9

interstitials = [
    fp.CellVariable(mesh=mesh, name="$e^-$", value=[0., impurity])
]

substitutionals = [
    fp.CellVariable(mesh=mesh, name="$M^+2$", value=[0., (Vs * "1 mol/l").inBaseUnits()]),
    fp.CellVariable(mesh=mesh, name="$A^-2$", value=[impurity, 0])
]

N = fp.CellVariable(mesh=mesh, name="$N$", value=[impurity, 0.])

interstitials[0].z   = -1
substitutionals[0].z = +2
substitutionals[1].z = -2
N.z                  = 0

components = interstitials + substitutionals + [N]

In [86]:
interstitials[0][0] = 0.
substitutionals[0][0] = 0.
X0 = [1., .5]
X0 = optimize.fsolve(solid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

interstitials[0][0] = X0[0]
substitutionals[0][0] = X0[1]


N[1] = 0.
substitutionals[1][1] = 0.
X0 = [1., 1.]
X0 = optimize.fsolve(liquid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

N[1] = X0[0]
substitutionals[1][1] = X0[1]

for Xj in components:
    Xj.muSL = fp.numerix.log(Xj[1] / Xj[0])
    print Xj.muSL, Xj.name, Xj

-20.3178007258 $e^-$ [  6.66666665e-01   1.00000000e-09]
-2.91877123242 $M^+2$ [ 0.33333333  0.018     ]
16.7058822881 $A^-2$ [  1.00000000e-09   1.79999995e-02]
20.6866018521 $N$ [  1.00000000e-09   9.64000000e-01]


## $X_\mathrm{impurity} = 10^{-18}$

In [87]:
impurity = 1e-18

interstitials = [
    fp.CellVariable(mesh=mesh, name="$e^-$", value=[0., impurity])
]

substitutionals = [
    fp.CellVariable(mesh=mesh, name="$M^+2$", value=[0., (Vs * "1 mol/l").inBaseUnits()]),
    fp.CellVariable(mesh=mesh, name="$A^-2$", value=[impurity, 0])
]

N = fp.CellVariable(mesh=mesh, name="$N$", value=[impurity, 0.])

interstitials[0].z   = -1
substitutionals[0].z = +2
substitutionals[1].z = -2
N.z                  = 0

components = interstitials + substitutionals + [N]

In [88]:
interstitials[0][0] = 0.
substitutionals[0][0] = 0.
X0 = [1., .5]
X0 = optimize.fsolve(solid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

interstitials[0][0] = X0[0]
substitutionals[0][0] = X0[1]


N[1] = 0.
substitutionals[1][1] = 0.
X0 = [1., 1.]
X0 = optimize.fsolve(liquid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

N[1] = X0[0]
substitutionals[1][1] = X0[1]

for Xj in components:
    Xj.muSL = fp.numerix.log(Xj[1] / Xj[0])
    print Xj.muSL, Xj.name, Xj

-41.0410665658 $e^-$ [  6.66666667e-01   1.00000000e-18]
-2.91877123242 $M^+2$ [ 0.33333333  0.018     ]
37.4291481528 $A^-2$ [  1.00000000e-18   1.80000000e-02]
41.4098676895 $N$ [  1.00000000e-18   9.64000000e-01]
